# Homework 4 - Programming Section Notebook

## Context

Given a dataset of bank client accounts and a target variable describing wether or not the client has a term deposit, create an unsupervised learning predictor through clustering of the data.

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset_file_path = "accounts.csv"

data_df = pd.read_csv(dataset_file_path)
data_df.drop_duplicates()

data_labels = data_df.columns
n_labels_to_keep = 8

#np.where(pd.isnull(data_df)) 
#np.where(pd.isna(data_df))
#both return 0, so no null or missing entries

X = data_df.drop(axis=1, labels=data_labels[n_labels_to_keep:])
X_data_types = X.dtypes

''' 

for label in data_labels[0:n_labels_to_keep]:
    if X_data_types[label] != 'int64':
        print(label)
        print(pd.get_dummies(X[label], drop_first=True, dtype='int64'))

 TODO: MAKE X COLUMNS ALL VALID TO BE NORMALIZED, BY CHANGING THE DTYPES WITHOUT LOSING INFORMATION       

'''
y = pd.get_dummies(data_df["deposit"], drop_first=True, dtype='int64') # 1 - "yes", 0 - "no"
y

job
       blue-collar  entrepreneur  housemaid  management  retired  \
0                0             0          0           0        0   
1                0             0          0           0        0   
2                0             0          0           0        0   
3                0             0          0           0        0   
4                0             0          0           0        0   
...            ...           ...        ...         ...      ...   
11157            1             0          0           0        0   
11158            0             0          0           0        0   
11159            0             0          0           0        0   
11160            0             0          0           0        0   
11161            0             0          0           0        0   

       self-employed  services  student  technician  unemployed  unknown  
0                  0         0        0           0           0        0  
1                  0         

,yes
0,1
1,1
2,1
3,1
4,1
...,...
11157,0
11158,0
11159,0
11160,0


## 1 - Clustering when normalizing the data using MinMaxScaler

In [46]:
from sklearn.preprocessing import MinMaxScaler

## MinMaxScaler Data Normalization
scaler = MinMaxScaler()
scaler.fit(X) 

ValueError: could not convert string to float: 'admin.'

## 2 - Clustering when normalizing the data using StandardScaler

In [ ]:
## StandardScaler Data Normalization